In [1]:
! pip3 install -q findspark
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext,SparkConf

In [3]:
conf=SparkConf().setAppName("test").setMaster("local[*]")
sc=SparkContext(conf=conf)
sc

<SparkContext master=local[*] appName=test>

In [4]:
#Note:
#Cluster name is test
#if i rerun the spark conf i will get error because of i can run only spark cluster
#so i need restart the kernel

In [5]:
#Note
#setMaster("local[4]") is represents the core and here 4 core is using
#setMaster("local[*]") is represents the core and here maximum number of core is using

In [6]:
sc.defaultParallelism

4

In [7]:
#Note:
# only 4 free core is available

### RDD

In [8]:
import random
randomlist=random.sample(range(0,40),10)
print(randomlist)

[3, 11, 33, 36, 34, 7, 20, 35, 21, 6]


In [9]:
#Create RDD
rdd1=sc.parallelize(randomlist,4)
rdd1

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:262

In [10]:
#Note:
#parallelize function is Partition

#Transformation function is not readable by using action we can readable

In [11]:
rdd1.collect()

[3, 11, 33, 36, 34, 7, 20, 35, 21, 6]

In [12]:
rdd1.getNumPartitions()

4

In [13]:
rdd1.glom().collect()

[[3, 11], [33, 36], [34, 7], [20, 35, 21, 6]]

In [14]:
rdd1.glom().take(2)

[[3, 11], [33, 36]]

In [15]:
rdd1.glom().collect()[3]

[20, 35, 21, 6]

In [16]:
rdd1.count()

10

In [17]:
rdd1.first()

3

In [18]:
rdd1.top(2)

[36, 35]

In [19]:
rdd1.distinct().collect()

[36, 20, 33, 21, 34, 6, 3, 11, 7, 35]

In [20]:
def myfunc(item):
    return (item+1)*3

rdd_map=rdd1.map(myfunc)
rdd_map.collect()

[12, 36, 102, 111, 105, 24, 63, 108, 66, 21]

### Map

In [21]:
rdd_map=rdd1.map(lambda item:item*3)
rdd_map.collect()

[9, 33, 99, 108, 102, 21, 60, 105, 63, 18]

### Filter

In [22]:
rdd_filter=rdd1.filter(lambda x : x%2 == 0)
rdd_filter.collect()

[36, 34, 20, 6]

In [23]:
rdd_filter.glom().collect()

[[], [36], [34], [20, 6]]

In [24]:
rdd_filter.count()

4

In [25]:
#Note
#Since it it value is 2 value we can reparition 


In [26]:
rdd_filter.repartition(1)

MapPartitionsRDD[22] at coalesce at NativeMethodAccessorImpl.java:0

In [27]:
rdd_filter.collect()

[36, 34, 20, 6]

In [28]:
# rdd1.collect()

### FilterMap

In [29]:
rdd_flatmap=rdd1.flatMap(lambda x:[x+2,x+5])
rdd_flatmap.collect()

[5, 8, 13, 16, 35, 38, 38, 41, 36, 39, 9, 12, 22, 25, 37, 40, 23, 26, 8, 11]

In [30]:
rdd_flatmap.glom().collect()

[[5, 8, 13, 16],
 [35, 38, 38, 41],
 [36, 39, 9, 12],
 [22, 25, 37, 40, 23, 26, 8, 11]]

### Reduce

In [31]:
rdd_flatmap.reduce(lambda x,y:x+y)

482

In [32]:
#Descriptive Statistics:

In [33]:
print(rdd1.max(),rdd1.min(),rdd1.mean(),rdd1.stdev(),rdd1.sum())

36 3 20.6 12.563439019631527 206


### MapPartitions

In [37]:
def myfunc(partition):
    sum=0
    for item in partition:
        sum=sum+item
    yield sum

In [38]:
rdd1.mapPartitions(myfunc).collect()

[14, 69, 41, 82]